# Neural Network on GPU



From Kaggle:
"MNIST ("Modified National Institute of Standards and Technology") is the de facto “hello world” dataset of computer vision. Since its release in 1999, this classic dataset of handwritten images has served as the basis for benchmarking classification algorithms. As new machine learning techniques emerge, MNIST remains a reliable resource for researchers and learners alike."

[Read more.](https://www.kaggle.com/c/digit-recognizer)


<a title="By Josef Steppan [CC BY-SA 4.0 (https://creativecommons.org/licenses/by-sa/4.0)], from Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:MnistExamples.png"><img width="512" alt="MnistExamples" src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets


## STEP 1: LOADING DATASET

In [ ]:
train_dataset = dsets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



## STEP 2: MAKING DATASET ITERABLE

In [ ]:
batch_size = 100
n_iters = 1200
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

print("Number of epochs: " + str(num_epochs))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


Number of epochs: 2


## STEP 3: CREATE MODEL CLASS

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(32 * 4 * 4, 10)

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)

        return out



## STEP 4: INSTANTIATE MODEL CLASS

In [ ]:
# Number of CUDA devices
# The first device is always named "cuda:0"
# The second one is "cuda:1", etc.
print(torch.cuda.device_count())

1


In [ ]:
model = CNNModel()

####################################################################
#  USE GPU FOR MODEL                                               #
#  The model must be put on the GPU before declaring the optimizer #
####################################################################

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#device = "cpu"

model.to(device)





CNNModel(
  (cnn1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=512, out_features=10, bias=True)
)

## STEP 5: INSTANTIATE LOSS CLASS

In [ ]:
criterion = nn.CrossEntropyLoss()



## STEP 6: INSTANTIATE OPTIMIZER CLASS

In [ ]:
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)



Function to compute the accuracy on the test set

### Question: modify the following code to exploit the GPU instead of the CPU

In [ ]:
def test_model(test_loader, model, device):
  # Calculate Accuracy
  correct = 0
  total = 0
  # Iterate through test dataset
  for images, labels in test_loader:
    #######################
    #  USE GPU FOR MODEL  #
    #######################
    images = images.to(device)
    labels = labels.to(device)

    # Forward pass only to get logits/output
    outputs = model(images)

    # Get predictions from the maximum value
    _, predicted = torch.max(outputs, 1)

    # Total number of labels
    total += labels.size(0)

    # Total correct predictions
    correct += (predicted == labels).sum()

  accuracy = 100 * float(correct) / float(total)

  return accuracy

## STEP 7: TRAIN THE MODEL

### Question: modify the following code to exploit the GPU instead of the CPU

In [ ]:
%%time
# Time execution of a Python statement or expression.
# wall time is the actual time taken from the start of a computer program to the end

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        #######################
        #  USE GPU FOR MODEL  #
        #######################
        images = images.to(device)
        labels = labels.to(device)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy on the test set
            accuracy = test_model(test_loader, model, device)

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy on test set: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 0.14945346117019653. Accuracy on test set: 95.18
Iteration: 1000. Loss: 0.14531609416007996. Accuracy on test set: 95.9
CPU times: user 13.9 s, sys: 71.6 ms, total: 14 s
Wall time: 14 s


### Question: compare the wall time on GPU to the wall time on CPU

In [ ]:
# device = "cpu" # and run again the notebook

### Question: increase the number of epoch until 5 to see if we can expect a better average accuracy

In [ ]:
# n_iters = 3000 # increase n_iters